In [1]:
## Importing Packages
import numpy as np
import pandas as pd
import random
from sklearn.metrics import accuracy_score,classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [14]:
## Data Loading
data = pd.read_csv("Bank_Personal_Loan_Modelling.csv")
data.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [15]:
x = data.drop(['Personal Loan', 'ID', 'ZIP Code'], axis=1)  ##Indepent Var
y = data['Personal Loan'].values ## Dependent var

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

In [17]:
## Feature Scalling
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [18]:
class NeuralNetwork:
    def __init__(self, layer_size, weights):
        self.input_size = layer_size[0]
        self.hidden1_size = layer_size[1]
        self.output_size = layer_size[2]

        self.weights = weights

        self.W1_size = self.input_size * self.hidden1_size
        self.W2_size = self.hidden1_size * self.output_size

        self.w1 = self.weights[:self.W1_size].reshape(self.input_size, self.hidden1_size)
        self.w2 = self.weights[self.W1_size: self.W1_size + self.W2_size].reshape(self.hidden1_size, self.output_size)


    def forwardPropagation(self, X):
        self.z2 = np.dot(X, self.w1)
        self.a2 = reLU(self.z2)

        self.z3 = np.dot(self.a2, self.w2)
        output = sigmoid(self.z3)

        return output

In [19]:
##Activation Functions
def sigmoid(z):
    return 1.0 / (1 + np.exp(-z))

def reLU(z):
    return np.maximum(0.0, z)

In [20]:
layer_size = [11, 8, 1]
tot_weights = layer_size[0] * layer_size[1] + layer_size[1] * layer_size[2]

In [21]:
## Fitness Function Definition
def fitness(weight):
    nn = NeuralNetwork(layer_size, weight)
    y_pred = nn.forwardPropagation(x_train)
    return accuracy_score(np.round(y_pred), y_train)

In [22]:
## EA Params
population = 500
epochs = 20
pool_size = 20

In [23]:
def init_population():
    weight_pop = []
    for _ in range(population):
        weight_pop.append(np.random.uniform(-2, 2, tot_weights))

    return np.array(weight_pop)

np.random.seed(7)
initial_population = init_population()

In [24]:
def random_selection(population):
    mating_pool = population[:pool_size]
    return mating_pool

In [25]:
def crossover(mating_pool):
    children = []

    while len(mating_pool):
        p1_index, p2_index = np.random.choice(mating_pool.shape[0], 2, replace=False)
        p1, p2 = mating_pool[p1_index, :], mating_pool[p2_index, :]
        r = random.uniform(0, 1)
        s = random.uniform(-1, 0)
        c1 = r * p1 + s * p2
        c2 = s * p2 + r * p1
        children.extend([c1, c2])
        mating_pool = np.delete(mating_pool, [p1_index, p2_index], axis=0)
    return np.array(children)

In [26]:
def mutation(children):
    mut_mat = np.random.uniform(0.75, 1.25, children.shape)
    children = mut_mat * children
    return children

In [27]:
def replace_population(child_population, current_population):
    next_population = np.append(current_population[:population - pool_size], child_population, axis=0)
    next_population = np.array(sorted(next_population, key=lambda x: fitness(x), reverse=True))
    return next_population

In [28]:
final = 0
fit_plt = []
current_population = np.array(sorted(initial_population, key=lambda x: fitness(x), reverse=True))
for i in range(epochs):
    mating_pool = random_selection(current_population)
    children = crossover(mating_pool)
    children = mutation(children)
    current_population = replace_population(children, current_population)
    final = current_population[0]
    final_fit = fitness(final)
    print(f"Best fitness of Generation {i + 1}: {final_fit}")
    fit_plt.append(final_fit)

Best fitness of Generation 1: 0.90625
Best fitness of Generation 2: 0.90625
Best fitness of Generation 3: 0.90625
Best fitness of Generation 4: 0.90625
Best fitness of Generation 5: 0.90625
Best fitness of Generation 6: 0.90625
Best fitness of Generation 7: 0.90625
Best fitness of Generation 8: 0.90625
Best fitness of Generation 9: 0.90625
Best fitness of Generation 10: 0.90625
Best fitness of Generation 11: 0.90775
Best fitness of Generation 12: 0.90775
Best fitness of Generation 13: 0.90775
Best fitness of Generation 14: 0.90875
Best fitness of Generation 15: 0.90875
Best fitness of Generation 16: 0.909
Best fitness of Generation 17: 0.909
Best fitness of Generation 18: 0.909
Best fitness of Generation 19: 0.909
Best fitness of Generation 20: 0.909


In [29]:
nn = NeuralNetwork(layer_size, final)
out = nn.forwardPropagation(x_test)
print(classification_report(np.round(out), y_test))

              precision    recall  f1-score   support

         0.0       1.00      0.90      0.95       996
         1.0       0.03      0.75      0.06         4

    accuracy                           0.90      1000
   macro avg       0.51      0.83      0.50      1000
weighted avg       1.00      0.90      0.94      1000

